### Get YouTube Audios - Start

In [22]:
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader
from pydub import AudioSegment
import os

# List of YouTube URLs
urls = ["https://www.youtube.com/watch?v=9eIMWmRG12w", "https://www.youtube.com/watch?v=Pg4w-rwTCXE"]
# urls = ["https://www.youtube.com/watch?v=qt-B2cg0pCM"]

# Directory to save audio files
save_dir = os.path.expanduser("~/Downloads/YouTube")

# Create a YoutubeAudioLoader instance
loader = YoutubeAudioLoader(urls, save_dir)

# Download and save audio files
for blob in loader.yield_blobs():
  if blob is not None and blob.data:  # Check if blob exists and has data
    audio_segment = AudioSegment.from_mp3(blob.data)
    filename = blob.metadata['title'] + ".mp3"  # Use title as filename
    audio_segment.export(os.path.join(save_dir, filename), format="mp3")

print(f"Audio files saved to: {save_dir}")

[youtube] Extracting URL: https://www.youtube.com/watch?v=9eIMWmRG12w
[youtube] 9eIMWmRG12w: Downloading webpage
[youtube] 9eIMWmRG12w: Downloading ios player API JSON
[youtube] 9eIMWmRG12w: Downloading player f8071a08


         n = ArIh65bF7CzwhOfYGuHa ; player = https://www.youtube.com/s/player/f8071a08/player_ias.vflset/en_US/base.js
         n = nxy-UDh1z0HL3SKsdNqW ; player = https://www.youtube.com/s/player/f8071a08/player_ias.vflset/en_US/base.js


[youtube] 9eIMWmRG12w: Downloading m3u8 information
[info] 9eIMWmRG12w: Downloading 1 format(s): 140
[download] Destination: C:\Users\ping\Downloads\YouTube\Sadhguru： Pencerahan Akhir Tahun ｜ Endgame #109 (Luminaries).m4a
[download] 100% of   87.02MiB in 00:00:05 at 17.20MiB/s    
[FixupM4a] Correcting container of "C:\Users\ping\Downloads\YouTube\Sadhguru： Pencerahan Akhir Tahun ｜ Endgame #109 (Luminaries).m4a"
[ExtractAudio] Not converting audio C:\Users\ping\Downloads\YouTube\Sadhguru： Pencerahan Akhir Tahun ｜ Endgame #109 (Luminaries).m4a; file is already in target format m4a
[youtube] Extracting URL: https://www.youtube.com/watch?v=Pg4w-rwTCXE
[youtube] Pg4w-rwTCXE: Downloading webpage
[youtube] Pg4w-rwTCXE: Downloading ios player API JSON
[youtube] Pg4w-rwTCXE: Downloading player 28fd7348


         n = P5BYq0cajk6BUkMAV ; player = https://www.youtube.com/s/player/28fd7348/player_ias.vflset/en_US/base.js
         n = 0Z-2ZpAIQW-IMSPQF ; player = https://www.youtube.com/s/player/28fd7348/player_ias.vflset/en_US/base.js


[youtube] Pg4w-rwTCXE: Downloading m3u8 information
[info] Pg4w-rwTCXE: Downloading 1 format(s): 140
[download] Destination: C:\Users\ping\Downloads\YouTube\Michael Levitt： Studied Physics, Masters Biology, Won Nobel in Chemistry ｜ Endgame #193 (Luminaries).m4a
[download] 100% of  125.33MiB in 00:00:09 at 12.65MiB/s    
[FixupM4a] Correcting container of "C:\Users\ping\Downloads\YouTube\Michael Levitt： Studied Physics, Masters Biology, Won Nobel in Chemistry ｜ Endgame #193 (Luminaries).m4a"
[ExtractAudio] Not converting audio C:\Users\ping\Downloads\YouTube\Michael Levitt： Studied Physics, Masters Biology, Won Nobel in Chemistry ｜ Endgame #193 (Luminaries).m4a; file is already in target format m4a
Audio files saved to: C:\Users\ping/Downloads/YouTube


### Get YouTube Audios - End

############

### Get YouTube Videos - Start

In [19]:
## Ref: https://github.com/pytube/pytube/issues/1973#issuecomment-2232907131

from pytube.innertube import _default_clients
from pytube import cipher
import re

_default_clients["ANDROID"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["ANDROID_EMBED"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS_EMBED"]["context"]["client"]["clientVersion"] = "19.08.35"
_default_clients["IOS_MUSIC"]["context"]["client"]["clientVersion"] = "6.41"
_default_clients["ANDROID_MUSIC"] = _default_clients["ANDROID_CREATOR"]



def get_throttling_function_name(js: str) -> str:
    """Extract the name of the function that computes the throttling parameter.

    :param str js:
        The contents of the base.js asset file.
    :rtype: str
    :returns:
        The name of the function used to compute the throttling parameter.
    """
    function_patterns = [
        r'a\.[a-zA-Z]\s*&&\s*\([a-z]\s*=\s*a\.get\("n"\)\)\s*&&\s*'
        r'\([a-z]\s*=\s*([a-zA-Z0-9$]+)(\[\d+\])?\([a-z]\)',
        r'\([a-z]\s*=\s*([a-zA-Z0-9$]+)(\[\d+\])\([a-z]\)',
    ]
    #logger.debug('Finding throttling function name')
    for pattern in function_patterns:
        regex = re.compile(pattern)
        function_match = regex.search(js)
        if function_match:
            #logger.debug("finished regex search, matched: %s", pattern)
            if len(function_match.groups()) == 1:
                return function_match.group(1)
            idx = function_match.group(2)
            if idx:
                idx = idx.strip("[]")
                array = re.search(
                    r'var {nfunc}\s*=\s*(\[.+?\]);'.format(
                        nfunc=re.escape(function_match.group(1))),
                    js
                )
                if array:
                    array = array.group(1).strip("[]").split(",")
                    array = [x.strip() for x in array]
                    return array[int(idx)]

    raise RegexMatchError(
        caller="get_throttling_function_name", pattern="multiple"
    )

cipher.get_throttling_function_name = get_throttling_function_name

In [20]:
from pytube import YouTube
import os

# List of YouTube URLs
urls = ["https://www.youtube.com/watch?v=9eIMWmRG12w", "https://www.youtube.com/watch?v=Pg4w-rwTCXE"]

# Directory to save videos
save_dir = os.path.expanduser("~/Downloads/YouTube")

# Function to download a video
def download_video(url, save_dir):
  try:
    yt = YouTube(url)
    video = yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution').desc().first()
    video.download(output_path=save_dir)
    print(f"Downloaded: {video.title}")
  except Exception as e:
    print(f"Error downloading video: {e}")

# Download videos
for url in urls:
  download_video(url, save_dir)


Downloaded: Sadhguru: Pencerahan Akhir Tahun | Endgame #109 (Luminaries)
Downloaded: Michael Levitt: Studied Physics, Masters Biology, Won Nobel in Chemistry | Endgame #193 (Luminaries)


### Get YouTube Videos - End

In [23]:
import os

def get_file_names(directory_path):
  """Gets the file names from a specified directory.

  Args:
    directory_path: The path to the directory.

  Returns:
    A list of file names.
  """

  file_names = []
  for root, _, files in os.walk(directory_path):
    for file in files:
      file_names.append(file)
  return file_names

# Example usage:
directory_path = save_dir
file_names = get_file_names(directory_path)
print(file_names)


['Michael Levitt： Studied Physics, Masters Biology, Won Nobel in Chemistry ｜ Endgame #193 (Luminaries).m4a', 'Sadhguru： Pencerahan Akhir Tahun ｜ Endgame #109 (Luminaries).m4a']


#### Took 30 minutes to transcribe 2:15:18 hours of audio on my PC using CPU

In [33]:
import whisper

for file_name in file_names[:1]:
  path_audio = save_dir + "/" + file_name
  print(path_audio)
  if os.path.exists(path_audio):
    print('file exists')
  else:  
    print('file does not exists')
  model = whisper.load_model("base")

  # result = model.transcribe(path_audio, language="fr")  # Transcribe in French
  result = model.transcribe(path_audio, language="en")  # Transcribe in English

  print(result["text"])

  # Generate subtitles
  print(result["segments"])  # List of segments with start, end, and text

C:\Users\ping/Downloads/YouTube/Michael Levitt： Studied Physics, Masters Biology, Won Nobel in Chemistry ｜ Endgame #193 (Luminaries).m4a
file exists


c:\Users\ping\Files_win10\python\py310\.venv\lib\site-packages\whisper\transcribe.py:115: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Biological intelligence is not us. We did not create biology. And therefore it's an alien for a intelligent, for much we can learn a great deal. AI has just hacked the operating system of human civilization. See if you look at the lesson of evolution, the great strength for the future survival of humanity is that diversity. My name is Michael Ebert. I'm a professor at Stanford Medical School. The Instructural Biology was an association also in computer science. Professor Michael Levit, recipient of the 2013 Nobel Prize in chemistry. He was interdisciplinary before it was fancy to be interdisciplinary. And he was interdisciplinary because the work and the vision that he had required that he master disciplines, from physics to chemistry to biology to computer science. My natural tendency in life has always been to treat life as a buffet. Take what you like and don't complain about what you don't like. And by that way you can always find good everywhere. We don't know the future. However

In [43]:
save_dir + "/Michael Levitt"

'C:\\Users\\ping/Downloads/YouTube/Michael Levitt'

In [45]:
# import os
import sys
sys.path.append('../..')
from py3810.myUtils import pickle_dump, pickle_load

pickle_dump(file_to_pickle=result["text"], filename_pickle="/Michael Levitt_text", path_pickle_dump=save_dir)
pickle_dump(file_to_pickle=result["segments"], filename_pickle="/Michael Levitt_segments", path_pickle_dump=save_dir)

In [48]:
print(f'len(result["text"]: {len(result["text"])}')
print(f'len(result["segments"]: {len(result["segments"])}')

len(result["text"]: 119786
len(result["segments"]: 2252


In [50]:
result["text"]

" Biological intelligence is not us. We did not create biology. And therefore it's an alien for a intelligent, for much we can learn a great deal. AI has just hacked the operating system of human civilization. See if you look at the lesson of evolution, the great strength for the future survival of humanity is that diversity. My name is Michael Ebert. I'm a professor at Stanford Medical School. The Instructural Biology was an association also in computer science. Professor Michael Levit, recipient of the 2013 Nobel Prize in chemistry. He was interdisciplinary before it was fancy to be interdisciplinary. And he was interdisciplinary because the work and the vision that he had required that he master disciplines, from physics to chemistry to biology to computer science. My natural tendency in life has always been to treat life as a buffet. Take what you like and don't complain about what you don't like. And by that way you can always find good everywhere. We don't know the future. Howeve

In [37]:
result["segments"]

[{'id': 0,
  'seek': 0,
  'start': 0.0,
  'end': 5.0,
  'text': ' Biological intelligence is not us. We did not create biology.',
  'tokens': [50364,
   13007,
   4383,
   7599,
   307,
   406,
   505,
   13,
   492,
   630,
   406,
   1884,
   14956,
   13,
   50614],
  'temperature': 0.0,
  'avg_logprob': -0.32065853118896487,
  'compression_ratio': 1.5551181102362204,
  'no_speech_prob': 0.23306311666965485},
 {'id': 1,
  'seek': 0,
  'start': 5.0,
  'end': 7.6000000000000005,
  'text': " And therefore it's an alien for a intelligent,",
  'tokens': [50614,
   400,
   4412,
   309,
   311,
   364,
   12319,
   337,
   257,
   13232,
   11,
   50744],
  'temperature': 0.0,
  'avg_logprob': -0.32065853118896487,
  'compression_ratio': 1.5551181102362204,
  'no_speech_prob': 0.23306311666965485},
 {'id': 2,
  'seek': 0,
  'start': 7.6000000000000005,
  'end': 9.0,
  'text': ' for much we can learn a great deal.',
  'tokens': [50744, 337, 709, 321, 393, 1466, 257, 869, 2028, 13, 50814],
